In [3]:
#!/usr/bin/python3
import re, os, requests, time, sys
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from PIL import Image
from io import BytesIO
from datetime import datetime
from IPython.display import clear_output

global downloaded_urls
downloaded_urls = []
progress_part = 0

In [4]:
def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))
    clear_output(wait = True)
    text = "Progress: [{0}]/{0} {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100, progress_part)
    print(text)

def update_progress_part(part):
    progress_part = part
    
def safe_get(url):
    try:
        return requests.get(url).text
    except:
        return None

In [21]:
def collect_pages(directory, url, recursive=True):
    update_progress(0.0)
    
    # Scan sub urls
    print("Scanning " + url)
    page = safe_get(url)
    soup = BeautifulSoup(page, 'html.parser')
    bag = []
    bag.append(url)
    
    if (recursive == True and page != None):
        for href in soup.find_all('a'):
            try:
                jump_link = str(href.get('href'))
                sub_url = None
    
                if (jump_link[:4] == 'http'):
                    sub_url = jump_link
                else:
                    # TODO: often have to modify /
                    pos = url[8:].index('/')
                    base_url = url[:pos+8] + '/'
                    sub_url = base_url + jump_link
                
                if sub_url not in bag:  
                    print("Adding " + sub_url)
                    bag.append(sub_url)
            except Exception:
                print('exception ' + str(Exception))
                continue

        # Iterate over all sites if recursive
        number_of_elements = len(bag)
        for i in range(number_of_elements):
            print("Scanning page from: " + bag[i])
            bag.append(bag[i])
            update_progress(i / number_of_elements)

    update_progress(1.0)
    return bag

def download_images(directory, keyword, urls):
    update_progress(0.0)
    number_of_elements = len(urls)
    for i in range(number_of_elements):
        download_img_src(directory, keyword, urls[i])
        update_progress(i / number_of_elements)
    update_progress(1.0)
    
def save_contents(directory, keyword, urls):
    update_progress(0.0)
    number_of_elements = len(urls)
    for i in range(number_of_elements):
        save_content(directory, keyword, urls[i], i)
        update_progress(i / number_of_elements)
    update_progress(1.0)

def save_content(directory, keyword, url, i):
    page = safe_get(url)
    if (page == None):
        return
    time_part = datetime.now().strftime('%Y%m%d%H%M%S')
    string = directory + '/content_' + keyword + '_' + time_part + '_' + str(i) + '.html'
    fp = open(string, 'wb')
    fp.write(bytes(page, 'utf-8'))
    fp.close()           

In [22]:
def download_img_src(directory, keyword, url):
    global downloaded_urls
    page = safe_get(url)
    if (page == None):
        return
    i=0
    soup = BeautifulSoup(page, 'html.parser')
    
    for raw_img in soup.find_all('img'):
        #print("Processing image: " + str(raw_img))
        try:
            link = raw_img.get('src')
            if (not (link == None)):
                final_url = None
                if (link[:4] == 'http'):
                    final_url = link
                else:
                    # TODO: often have to modify /
                    pos = url[8:].index('/')
                    base_url = url[:pos+8] #+ '/'
                    final_url = base_url + link
                    
                if (final_url not in downloaded_urls):
                    time_part = datetime.now().strftime('%Y%m%d%H%M%S')
                    try: 
                        pic = requests.get(final_url, timeout=10)    
                        string = directory + '/pic_' + keyword + '_' + time_part + '_' + str(i) + final_url[len(final_url)-4:]
                        fp = open(string, 'wb')
                        fp.write(pic.content)
                        fp.close()
                        downloaded_urls.append(final_url)
                        i=i+1
                    except:
                        continue
        except:
            print('exception' + str(Exception))
            continue

In [ ]:
# Building classifier for cheking innapropiate photos
pages = collect_pages("source", 'https://www.textures.com/', True)
#pages = collect_pages("source", 'http://www.nukleus.nu/VersusVote.php/', True)

for i in range(2,300):
    pages = collect_pages("source", 'https://www.textures.com/browse/rubber/' + str(i), True)
    download_images('source', 'floors', pages)

Progress: [######--------------]/######-------------- 31.0%


In [ ]:
#print(pages)
download_images('source', 'logos', pages)
#save_contents('source', 'nicks-scene', pages)
#print("end")